# Step 4: Test and Compare Models

In this step, you'll test your trained model and compare it with EasyOCR's pre-trained model to evaluate performance.

## What You'll Learn:
- How to set up a custom OCR model for inference
- How to compare custom vs pre-trained models
- How to interpret OCR results and confidence scores

## 4.1 Setup: Copy Trained Model

First, copy your trained model to the `user_network_dir` directory where EasyOCR will load it.

**Important Files Needed:**
- `user_network_dir/custom.pth` - Your trained model weights
- `user_network_dir/custom.yaml` - Model configuration (architecture, character list)
- `user_network_dir/custom.py` - Model architecture definition

In [1]:
from pathlib import Path
import shutil

# Create the user network directory if it doesn't exist
user_network_dir = Path("./user_network_dir")
user_network_dir.mkdir(parents=True, exist_ok=True)

# Copy your trained model to the user network directory
# ⚠️ IMPORTANT: Change the folder name below to match your training run
source_model = Path("./saved_models/TPS-VGG-BiLSTM-CTC-Seed1111-20251119-024440/best_accuracy.pth")
target_model = user_network_dir / "custom.pth"

if not source_model.exists():
    raise FileNotFoundError(f"Expected trained model at {source_model} but it was not found.")

shutil.copy2(source_model, target_model)
print(f"Copied {source_model} -> {target_model}")

Copied saved_models\TPS-VGG-BiLSTM-CTC-Seed1111-20251119-024440\best_accuracy.pth -> user_network_dir\custom.pth


## 4.2 Prepare Test Images

Place your test images in the `demo_images` folder. These should be images similar to what your model will process in production (e.g., receipts, invoices, signs).

```bash
demo_images/
  ├── demo_image_01.jpg
  ├── demo_image_02.jpg
  └── ...
```

## 4.3 Test with Pre-trained Model (Baseline)

First, test with EasyOCR's pre-trained model to establish a baseline performance.

In [2]:
%pip install easyocr

  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scipy-1.16.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scikit_image-0.25.2-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached python_bidi-0.6.7-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached shapely-2.1.2-cp312-cp312-win_amd64.whl.metadata (7.1 kB)
  Using cached ninja-1.13.0-py3-none-win_amd64.whl.metadata (5.1 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached imageio-2.37.2-py3-none-any.whl.metadata (9.7 kB)
  Using cached tifffile-2025.10.16-py3-none-any.whl.metadata (31 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
Using cached ninja-1.13.0-py3-none-win_amd64.whl (309 kB)
Using cached opencv_python_

In [2]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with pre-trained traditional Chinese model
print("Loading EasyOCR pre-trained model...")
reader_pretrained = Reader(['ch_tra'], gpu=True)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_pretrained.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading EasyOCR pre-trained model...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete
TESTING WITH EASYOCR PRE-TRAINED MODEL (Baseline)

filename: 'demo_image_01.jpg', confidence: 0.5216, string: '九龍青衣細山路2號美景花園平臺72號舖'


## 4.4 Test with Your Custom Model

Now test with your trained model. Make sure you have:
- ✅ `user_network_dir/custom.pth` (model weights)
- ✅ `user_network_dir/custom.yaml` (configuration)
- ✅ `user_network_dir/custom.py` (architecture)

In [1]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize EasyOCR with your custom model
print("Loading your custom trained model...")
reader_custom = Reader(
    ['ch_tra'], 
    gpu=True,
    model_storage_directory='./user_network_dir',
    user_network_directory='./user_network_dir',
    recog_network='custom'
)

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*70}")
print("TESTING WITH YOUR CUSTOM MODEL")
print(f"{'='*70}\n")

# Test each image - simplified version
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    results = reader_custom.readtext(file_path)
    
    for bbox, text, confidence in results:
        print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, text))

Loading your custom trained model...

TESTING WITH YOUR CUSTOM MODEL

filename: 'demo_image_01.jpg', confidence: 0.0124, string: '如湯眼飯'


## 4.5 Side-by-Side Comparison

Compare both models on the same images to see which performs better.

In [4]:
from easyocr import Reader
import os

# Set GPU devices
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Initialize both models
print("Loading both models...")
reader_pretrained = Reader(['ch_tra'], gpu=True)
reader_custom = Reader(['ch_tra'], gpu=True,
                      model_storage_directory='./user_network_dir',
                      user_network_directory='./user_network_dir',
                      recog_network='custom')

# Get test images
demo_images_dir = './demo_images'
test_files = [f for f in os.listdir(demo_images_dir) if not f.startswith('.')]
test_files.sort()

print(f"\n{'='*80}")
print("SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model")
print(f"{'='*80}\n")

# Compare results for each image
for filename in test_files:
    file_path = os.path.join(demo_images_dir, filename)
    print(f"📷 Testing: {filename}")
    print(f"{'-'*80}")
    
    # Test with pre-trained model
    results_pretrained = reader_pretrained.readtext(file_path)
    print("🔵 EasyOCR Pre-trained Model:")
    if results_pretrained:
        for bbox, text, confidence in results_pretrained:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print()
    
    # Test with custom model
    results_custom = reader_custom.readtext(file_path)
    print("🟢 Your Custom Model:")
    if results_custom:
        for bbox, text, confidence in results_custom:
            print("   confidence: %.4f, string: '%s'" % (confidence, text))
    else:
        print("   No text detected")
    
    print(f"\n{'='*80}\n")

Loading both models...

SIDE-BY-SIDE COMPARISON: Pre-trained vs Custom Model

📷 Testing: demo_image_01.jpg
--------------------------------------------------------------------------------
🔵 EasyOCR Pre-trained Model:
   confidence: 0.0148, string: '九龍青衣紐山路2號美景花園平壼72號舖'

🟢 Your Custom Model:
   confidence: 0.0356, string: 't  l $謳'




## 4.6 Test with Benchmark demo.py (Alternative Method)

Use the `deep-text-recognition-benchmark/demo.py` script directly to test your trained RCNN model. This bypasses EasyOCR integration and uses the original benchmark inference pipeline.

**When to use this:**
- Your model was trained with the benchmark repository (not EasyOCR)
- You want to avoid EasyOCR `custom` network setup complexity
- Quick testing without state_dict alignment issues

**Note:** Make sure the `--Transformation`, `--FeatureExtraction`, `--SequenceModeling`, and `--Prediction` arguments match exactly what you used during training.

In [ ]:
!python deep-text-recognition-benchmark/demo.py \
    --Transformation TPS \
    --FeatureExtraction RCNN \
    --SequenceModeling BiLSTM \
    --Prediction CTC \
    --image_folder demo_images \
    --saved_model saved_models/TPS-RCNN-BiLSTM-CTC-Seed1111-20251202-205307/best_accuracy.pth